In [1]:
"""qsvd-v4.py"""

# A = Original matrix
# B = Square root of Matrix A
# U, V, L = Matrices obtained after SVD
# Ut, Vt, Lt = Truncated U, V, L matrices
# Bnp = Matrix obtained from U, V, L with no phase added or truncation done
# Bp = Matrix obtained from U, V, L after truncation and phase added
# Bt = Matrix obtained from U, V, L after truncation, no phase added

""" Import Libraries """
import numpy as np
import scipy.optimize as optimize

### Method 1

In [2]:
""" Optimization Algorithm """
""" New Matrix """
def newMat_2(x, Ut,Lt,Vt):
    l0,l1 = Lt[0], Lt[1]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    for i in range(Ut.shape[0]):
      U_new[i][0] = np.cos(x[i]) / l0
      U_new[i][1] = (np.sin(x[i]) / l1) * np.exp(1j*x[i+Ut.shape[0]])
    return np.dot(np.dot(U_new,np.diag(Lt)), Vt)


def newMat_3(x, Ut,Lt,Vt):
    l0,l1,l2 = Lt[0], Lt[1], Lt[2]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    for i in range(Ut.shape[0]):
      U_new[i][0] = np.cos(x[i]) / l0
      U_new[i][1] = (np.sin(x[i]) / l1) * (np.cos(x[i+Ut.shape[0]])) * np.exp(1j*x[i+2*Ut.shape[0]])
      U_new[i][2] = (np.sin(x[i]) / l2) * (np.sin(x[i+Ut.shape[0]])) * np.exp(1j*x[i+3*Ut.shape[0]])
    return np.dot(np.dot(U_new,np.diag(Lt)), Vt)


def newMat_4(x, Ut,Lt,Vt):
    l0,l1,l2,l3 = Lt[0], Lt[1], Lt[2], Lt[3]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    for i in range(Ut.shape[0]):
      U_new[i][0] = (np.cos(x[i]) / l0) * (np.cos(x[i+Ut.shape[0]]))
      U_new[i][1] = (np.cos(x[i]) / l1) * (np.sin(x[i+Ut.shape[0]])) * np.exp(1j*x[i+3*Ut.shape[0]])
      U_new[i][2] = (np.sin(x[i]) / l2) * (np.cos(x[i+2*Ut.shape[0]])) * np.exp(1j*x[i+4*Ut.shape[0]])
      U_new[i][3] = (np.sin(x[i]) / l3) * (np.sin(x[i+2*Ut.shape[0]])) * np.exp(1j*x[i+5*Ut.shape[0]])
    return np.dot(np.dot(U_new,np.diag(Lt)), Vt)


""" Cost Function """
def costFn(x, Ut,Lt,Vt,B,k):
    if k==2:
      Bp = newMat_2(x, Ut, Lt, Vt)
    elif k==3:
      Bp = newMat_3(x, Ut, Lt, Vt)
    elif k==4:
      Bp = newMat_4(x, Ut, Lt, Vt)
    else:
      return 0
    loss = np.linalg.norm(B**2 - np.abs(Bp)**2)
    return (loss)


""" Calculate Results """
def calcResults(k):
  print("k = ", str(k))
  m,n = 5,5
  print("m = ",m,", n = ",n)
  res = np.zeros((100,2))
  for i in range(100):
      A = np.random.rand(m, n)
      for j in range(m): A[j] /= sum(A[j])
      B = np.sqrt(A)
      U, L, V = np.linalg.svd(B, full_matrices=False)
      initial_guess = np.ones((2*m*(k-1),), dtype=np.longdouble)
      Ut = U[:, :k]
      Vt = V[:k]
      Lt = L[:k]
      Bt = np.dot(np.dot(Ut,np.diag(Lt)), Vt)
      result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut,Lt,Vt,B,k),
                                  tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
      res[i][0] = (np.linalg.norm(B**2 - Bt**2))
      res[i][1] = costFn(result.x,Ut,Lt,Vt,B,k)
      if(i%10==0):
        print(i, " ", end='')
  print('\n', res)
  print("\nResult:", res.mean(axis=0))
  return

In [3]:
calcResults(k=2)

k =  2
m =  5 , n =  5
0  10  20  30  40  50  60  70  80  90  
 [[0.28899252 0.25339339]
 [0.17288546 0.15883213]
 [0.36699881 0.35296253]
 [0.3156626  0.25044257]
 [0.28141211 0.25265156]
 [0.39096132 0.34940413]
 [0.30033708 0.27339973]
 [0.24772962 0.23129339]
 [0.27635517 0.26622169]
 [0.20204975 0.17697469]
 [0.40293986 0.36941763]
 [0.46149213 0.39713142]
 [0.23591406 0.2004669 ]
 [0.34543448 0.29562948]
 [0.36610041 0.33254991]
 [0.35480765 0.30865379]
 [0.22713919 0.19695336]
 [0.47248588 0.45573881]
 [0.2554051  0.22853355]
 [0.17433037 0.16943629]
 [0.35624578 0.33836657]
 [0.27719747 0.21963396]
 [0.16551853 0.13742898]
 [0.46346556 0.36872248]
 [0.3688581  0.32671692]
 [0.22212656 0.18515146]
 [0.24475823 0.23022043]
 [0.26770844 0.20047417]
 [0.28877587 0.20105429]
 [0.24540316 0.23963563]
 [0.30237668 0.28755697]
 [0.43457834 0.38186591]
 [0.2766814  0.23695554]
 [0.40185493 0.34374258]
 [0.33923841 0.3255814 ]
 [0.20868176 0.17475446]
 [0.37963304 0.34542763]
 [0.1102123

In [4]:
calcResults(k=3)

k =  3
m =  5 , n =  5
0  10  20  30  40  50  60  70  80  90  
 [[0.14999308 0.08173275]
 [0.22491265 0.19842485]
 [0.22329515 0.14849974]
 [0.20009757 0.14056828]
 [0.08874077 0.08102036]
 [0.15840928 0.1157183 ]
 [0.19796656 0.14079722]
 [0.26030887 0.17065659]
 [0.28452867 0.19649521]
 [0.13233301 0.08942571]
 [0.15205167 0.11776241]
 [0.13251225 0.02171319]
 [0.22373584 0.16480592]
 [0.14632182 0.11234019]
 [0.25842646 0.12902122]
 [0.26006038 0.163214  ]
 [0.14597685 0.07635598]
 [0.12682264 0.11801786]
 [0.16371351 0.12565791]
 [0.11711977 0.08038289]
 [0.2145103  0.08151482]
 [0.11724871 0.12450191]
 [0.18685809 0.14180023]
 [0.24108706 0.19056444]
 [0.09963202 0.07051524]
 [0.17910467 0.11144123]
 [0.10746954 0.07691417]
 [0.11571123 0.08548047]
 [0.17490871 0.13473101]
 [0.29304872 0.17935014]
 [0.2069062  0.13812368]
 [0.02120292 0.01696553]
 [0.27601491 0.21252936]
 [0.13615471 0.06167852]
 [0.21112931 0.143145  ]
 [0.28198868 0.17080841]
 [0.10981322 0.05443949]
 [0.1097322

### Method 2

In [9]:
""" Optimization Algorithm """
""" New Matrix """
def newMat(x, Ut, Lt, Vt, k):
  U_new = np.zeros((Ut.shape), dtype=np.cfloat)
  if k==2:
    l0,l1 = Lt[0], Lt[1]
    U_new[0] = np.cos(x[0]) / l0
    U_new[1] = (np.sin(x[0]) / l1) * np.exp(1j*x[1])
  elif k==3:
    l0,l1,l2 = Lt[0], Lt[1], Lt[2]
    U_new[0] = np.cos(x[0]) / l0
    U_new[1] = (np.sin(x[0]) / l1) * (np.cos(x[1])) * np.exp(1j*x[2])
    U_new[2] = (np.sin(x[0]) / l2) * (np.sin(x[1])) * np.exp(1j*x[3])
  else:
    l0,l1,l2,l3 = Lt[0], Lt[1], Lt[2], Lt[3]
    U_new[0] = (np.cos(x[0]) / l0) * (np.cos(x[1]))
    U_new[1] = (np.cos(x[0]) / l1) * (np.sin(x[1])) * np.exp(1j*x[3])
    U_new[2] = (np.sin(x[0]) / l2) * (np.cos(x[2])) * np.exp(1j*x[4])
    U_new[3] = (np.sin(x[0]) / l3) * (np.sin(x[2])) * np.exp(1j*x[5])
  return U_new


""" Cost Function """
def costFn(x, Ut, Lt, Vt, B, k):
    U_new = newMat(x, Ut, Lt, Vt, k)
    Bp = np.dot(np.dot(U_new,np.diag(Lt)), Vt) 
    loss = np.linalg.norm(B**2 - np.abs(Bp)**2)
    return (loss)

""" Calculate Results """
def calcResults(k, a=0, b=0, c=0, d=0):
  print("k = ", str(k))
  df = []
  m,n = 5,5
  print("m = ",m,", n = ",n)
  res = np.zeros((100, 2))
  for i in range(100):
    A = np.random.rand(m, n)
    for j in range(m): A[j] /= sum(A[j])
    B = np.sqrt(A)
    U, L, V = np.linalg.svd(B, full_matrices=False)
    initial_guess = np.ones((2*(k-1),), dtype=np.longdouble)
    Ut = U[:, :k]
    Vt = V[:k]
    Lt = L[:k]
    Bt = np.dot(np.dot(Ut,np.diag(Lt)), Vt)
    U_new = np.zeros(Ut.shape, dtype=np.cfloat)
    for row in range(Ut.shape[0]):
      result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut[row, :],Lt,Vt,B[row,:],k),
                                tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
      U_new[row,:] = newMat(result.x, Ut[row, :],Lt,Vt,k)
    
    Bp = np.dot(np.dot(U_new,np.diag(Lt)), Vt) 
    
    res[i][0] = (np.linalg.norm(B**2 - Bt**2))
    res[i][1] = (np.linalg.norm(B**2 - ((Bp)*np.conj(Bp))))
    if i%10==0: print(i, end=' ')
  print('\n', res)
  print("\nResult:", res.mean(axis=0))
  return

In [10]:
calcResults(k=2)

k =  2
m =  5 , n =  5
0 10 20 30 40 50 60 70 80 90 
 [[0.21954744 0.16423274]
 [0.32384433 0.25975598]
 [0.31895125 0.25898875]
 [0.33209366 0.31275111]
 [0.30913059 0.27685922]
 [0.16664524 0.16127254]
 [0.3235222  0.30136741]
 [0.31920335 0.30623647]
 [0.35818618 0.3353103 ]
 [0.20463938 0.17279382]
 [0.26450313 0.21222905]
 [0.34384119 0.28415425]
 [0.36497283 0.33289575]
 [0.48021437 0.43493239]
 [0.41774142 0.37948045]
 [0.28941952 0.26383376]
 [0.36232998 0.2950808 ]
 [0.14137335 0.11830657]
 [0.32474503 0.29735964]
 [0.47026432 0.35854028]
 [0.25053721 0.23170457]
 [0.19449263 0.15885077]
 [0.30777508 0.26771131]
 [0.43459677 0.24467838]
 [0.37631503 0.31319603]
 [0.22576576 0.20592494]
 [0.32517086 0.3018826 ]
 [0.39009259 0.33311004]
 [0.34807927 0.33627681]
 [0.43783272 0.39808562]
 [0.44019944 0.36894028]
 [0.36615588 0.31015654]
 [0.31885609 0.24035784]
 [0.39347529 0.31722057]
 [0.25765869 0.24198851]
 [0.2986805  0.27912935]
 [0.34822621 0.32529713]
 [0.37952674 0.382511

In [11]:
calcResults(k=3)

k =  3
m =  5 , n =  5
0 10 20 30 40 50 60 70 80 90 
 [[2.67693146e-01 1.81516202e-01]
 [2.25738344e-01 9.55411237e-02]
 [2.08475314e-01 1.47437101e-01]
 [1.01057051e-01 2.96043511e-02]
 [2.84351167e-01 2.08913824e-01]
 [2.29964733e-01 2.00708021e-01]
 [1.10979845e-01 7.50925591e-02]
 [1.48934934e-01 1.09450146e-01]
 [1.58552840e-01 1.28097401e-01]
 [8.03561964e-02 4.32917972e-02]
 [2.32861489e-01 7.44773588e-02]
 [1.38938846e-01 8.61587816e-02]
 [2.49480690e-01 1.50850995e-01]
 [1.52163850e-01 7.13748690e-02]
 [1.15697831e-01 1.01218847e-01]
 [2.20555977e-01 1.63038003e-01]
 [1.12642085e-01 7.83584865e-02]
 [1.49558216e-01 1.43426205e-02]
 [2.38954012e-01 1.35258212e-01]
 [1.95425407e-01 7.66947213e-02]
 [1.09641095e-01 8.62555986e-02]
 [2.74462961e-01 2.02059274e-01]
 [7.35848089e-02 4.31788805e-02]
 [1.58293018e-01 7.11738228e-09]
 [1.75299873e-01 6.16710949e-02]
 [2.86859522e-01 2.26630903e-01]
 [5.60733498e-02 3.96375724e-02]
 [1.63161158e-01 8.06878935e-02]
 [1.26948167e-01 8.620